# Basics of TensorFlow

This tutorial starts out with very basic tensorflow examples and slowly works up.

In [3]:
import tensorflow as tf

## Starting Simple

Tensorflow constants and variables define a graph with symbolic expressions.

In [4]:
x = tf.constant(10)
y = tf.constant(5)

with tf.Session() as sess:
    result = sess.run(x + y)
    print(result)

15


If you look at results without being in the context of a session, you'll see
that expressions like `add`, `mul`, `sub`, `matmul`, etc. haven't been computed
yet.

In [5]:
x = tf.constant(10)
y = tf.constant(5)
z = tf.add(x, y)

print(z)

Tensor("Add:0", shape=(), dtype=int32)


But if you invoke an operation using run on a session object, it will execute
all steps necessary to compute that value.

In [6]:
with tf.Session() as sess:
    result = sess.run(z)
    print(result)

15


## Using Placeholders

Running constant operations isn't particularly exciting. But TensorFlow uses `placeholder` variables
that we can use to feed in values as inputs to the computational graph.

In [7]:
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
z = tf.add(x, y)

with tf.Session() as sess:
    result = sess.run(z, feed_dict={x: 3, y: 8})
    print(result)

11


We can feed in different values each time we execute the graph.

In [8]:
with tf.Session() as sess:
    result = sess.run(z, feed_dict={x: 4, y: 12})
    print(result)

16


Moreover, we can use Python's control structures to execute a graph multiple times and feed
different values each time.

In [9]:
from itertools import product

with tf.Session() as sess:
    for a, b in product(range(4), range(5, 8)):
        result = sess.run(z, feed_dict={x: a, y: b})
        print(result)

5
6
7
6
7
8
7
8
9
8
9
10


### Matrix Multiplication

Of course, TensorFlow has support for more than basic scalar operations (as you might guess from its name).

In [10]:
import numpy as np

In [11]:
x = tf.placeholder(tf.float32, shape=(2, 2))
y = tf.placeholder(tf.float32, shape=(2, 2))
z = tf.matmul(x, y)

_Note_: feed_dict values have to be variables because mutating them must be supported.

In [12]:
with tf.Session() as sess:
    a = np.array([[1, 2],
                  [3, 4]], dtype=np.float32)
    b = np.array([[1, 2],
                  [3, 4]], dtype=np.float32)  
    result = sess.run(z, feed_dict={y: a, x: b})
    print(result)

[[  7.  10.]
 [ 15.  22.]]


## A Simple Learning Example

We'll just train the network to learn how to scale and add!

In [68]:
f = lambda x: (x * 2) + 3

In [121]:
def batch_apply(fn, batch, scale=10):
    while True:
        val = np.random.rand(batch) * scale
        yield val, fn(val)

In [122]:
scale_add = batch_apply(f, 5)
next(scale_add)

(array([ 9.42712483,  4.30674698,  5.88916466,  9.77930526,  9.70053852]),
 array([ 21.85424965,  11.61349397,  14.77832932,  22.55861053,  22.40107704]))

### Model Hyperparameters

These are some global settings we can use to configure how we run and optimize our network.

In [133]:
training_epochs = 5001
batch_size = 32

This is a basic fitting function we'll use. Note that it follows the structure of our data generating
function exactly! This is a good toy example, but most functions won't work this way.

### Variable

We're also introducing the use of the TensorFlow `Variable`. How does it differ from constants or placeholders?

* Variables get updated by optimizers.
  * More formally, variables represent the _parameters_ of the model.
* Variables have to be initialized.

In [134]:
X = tf.placeholder(tf.float32, [batch_size])
y = tf.placeholder(tf.float32, [batch_size])

W = tf.Variable(1.0, [batch_size], name="weight")
b = tf.Variable(1.0, [batch_size], name="bias")

pred = tf.add(tf.mul(X, W), b)

We have to have a cost function and an optimizer that tries to change the weights and biases to
change the output of the cost function in the way we define (e.g. reduce it).

In [135]:
cost = tf.reduce_sum(tf.abs(tf.sub(pred, y)))
optimizer = tf.train.AdamOptimizer().minimize(cost)
init = tf.initialize_all_variables()

### Optimizing the model

To optimize the model, we run a loop that feeds in values to train the function.

* Note that `sess.run` is invoked with the optimizer as the first argument.
* Note also we're supply training data and labels in batches during each run.

In [137]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        train_X, train_y = next(batch_apply(f, batch_size))
        
        sess.run(optimizer, feed_dict={X: train_X, y: train_y})
        current_cost = sess.run(cost, feed_dict={X: train_X, y: train_y})

        if not (epoch % 1000):
            print("Epoch:",  '%04d' % (epoch+1),
                  "Cost:",   "{:.9f}".format(current_cost),
                  "Weight:", sess.run(W),
                  "Bias:",   sess.run(b))

Epoch: 0001 Cost: 229.017883301 Weight: 1.001 Bias: 1.001
Epoch: 1001 Cost: 32.076526642 Weight: 1.99936 Bias: 2.00105
Epoch: 2001 Cost: 5.378720284 Weight: 2.06642 Bias: 2.54869
Epoch: 3001 Cost: 0.002993822 Weight: 2.00003 Bias: 2.99992
Epoch: 4001 Cost: 0.035828114 Weight: 1.99984 Bias: 2.99962
Epoch: 5001 Cost: 0.003920317 Weight: 1.99999 Bias: 3.00017


### Learn to invert it!

Just by inverting the train and label values, we can teach the graph/model to learn
the inverse of our previous function.

In [139]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        ## Reversed order of _y and _X here is the only change!
        train_y, train_X = next(batch_apply(f, batch_size))
        
        sess.run(optimizer, feed_dict={X: train_X, y: train_y})
        current_cost = sess.run(cost, feed_dict={X: train_X, y: train_y})

        if not (epoch % 1000):
            print("Epoch:",  '%04d' % (epoch+1),
                  "Cost:",   "{:.9f}".format(current_cost),
                  "Weight:", sess.run(W),
                  "Bias:",   sess.run(b))

Epoch: 0001 Cost: 300.109436035 Weight: 0.999 Bias: 0.999
Epoch: 1001 Cost: 15.240823746 Weight: 0.395135 Bias: 0.236151
Epoch: 2001 Cost: 15.201331139 Weight: 0.427919 Bias: -0.32019
Epoch: 3001 Cost: 4.876611233 Weight: 0.472004 Bias: -1.05191
Epoch: 4001 Cost: 0.105165169 Weight: 0.49975 Bias: -1.50005
Epoch: 5001 Cost: 0.001037925 Weight: 0.500007 Bias: -1.50008
